## This notebook is solely to be able to perform the shuffling analysis on a separate notebook

In [1]:
# Downloaded libraries
from matplotlib import pyplot as plt
import numpy as np
import pynwb

# Own libraries
import main

In [2]:
directory = "data\TestPlitt"

# # SHUFFLE RUN 0
# handledFiles = ["sub-F1_ses-20190407T210000_behavior+ophys_small.nwb",\
#                "sub-F2_ses-20190417T210000_behavior+ophys_small.nwb",\
#                "sub-R1_ses-20190211T210000_behavior+ophys_small.nwb",\
#                "sub-R1_ses-20190223T210000_behavior+ophys_small.nwb"]

# # SHUFFLE RUN 1
# handledFiles = ["sub-F2_ses-20190407T210000_behavior+ophys_small.nwb",\
#                "sub-F5_ses-20190418T210000_behavior+ophys_small.nwb",\
#                "sub-R2_ses-20190209T210000_behavior+ophys_small.nwb",\
#                "sub-R2_ses-20190221T210000_behavior+ophys_small.nwb"]

# # SHUFFLE RUN 2
# handledFiles = ["sub-F2_ses-20190407T210000_behavior+ophys_small.nwb",\
#                "sub-F1_ses-20190416T210000_behavior+ophys_small.nwb",\
#                "sub-R3_ses-20190211T210000_behavior+ophys_small.nwb",\
#                "sub-R3_ses-20190223T210000_behavior+ophys_small.nwb"]

# SHUFFLE RUN 3
handledFiles = ["sub-F5_ses-20190407T210000_behavior+ophys_small.nwb",\
               "sub-F2_ses-20190416T210000_behavior+ophys_small.nwb",\
               "sub-R4_ses-20190824T210000_behavior+ophys_small.nwb",\
               "sub-R4_ses-20190905T210000_behavior+ophys_small.nwb"]

In [3]:
nNeurons = {}
deconvTraces = {}
startIndices = {}
baseMorph = {}
position = {}
nTrials = {}

readFiles = [main.getSessionHandle(file, directory) for file in handledFiles]
nFiles = len(readFiles)

for i, file in enumerate(readFiles):
    data = main.dataExtraction(readFiles[i].read())

    # nFrames = data["nFrames"]
    nNeurons[i] = data["nNeurons"]
    deconvTraces[i] = data["deconvTraces"]
    # tstartData = data["tstartData"]
    nTrials[i] = data["nTrials"]
    startIndices[i] = data["startIndices"]
    baseMorph[i] = data["baseMorph"]
    # totalMorph = data["totalMorph"]
    position[i] = data["position"]

In [4]:
import datetime
current_time = datetime.datetime.now()
print("Current date and time:", current_time)

Current date and time: 2023-08-12 18:29:48.590697


In [5]:
dbs = {}
pbs = {}
shifted_pbs = {}

df = {}
occp = {}

baseMorphList = {i:baseMorph[i][startIndices[i]] for i in range(nFiles)} 

nShuffles = 100

# SIMatrix = {i:np.zeros((nNeurons[i], 5)) for i in range(nFiles)}
# baseMorphList = {i:baseMorph[i][startIndices[i]] for i in range(nFiles)} 

# This will store the mega arrays off all SIs for all shuffles for each handled file
allSIs = {i:[] for i in range(nFiles)}

for iteration in range(nShuffles):
    SIMatrix = {i:np.zeros((nNeurons[i], 5)) for i in range(nFiles)}
    for i in range(nFiles):
        dbs[i], pbs[i] = main.trialize(deconvTraces[i], position[i], startIndices[i])
        shift_amounts = np.random.rand(nTrials[i], nNeurons[i])
        shifted_dbs = []
        for trial in range(nTrials[i]):
            nFrames = len(dbs[i][trial])
            shuffleIndex = np.tile(np.arange(nFrames), nNeurons[i]).reshape(-1,nFrames)
            shuffleIndex += (nFrames * shift_amounts[trial].reshape(-1, 1)).astype(int)
            shuffleIndex = (shuffleIndex % nFrames).astype(int)
            shifted_dbs.append(dbs[i][trial].T[np.arange(nNeurons[i])[:, None], shuffleIndex].T)
            
            
        df[i], occp[i] = main.positionalBin(shifted_dbs, pbs[i])
        del shifted_dbs
 
        for n in range(nNeurons[i]):
            SIMatrix[i][n,:] = main.getSpatialInformation(df[i][:,:,n], occp[i], baseMorphList[i])
        
        allSIs[i].append(SIMatrix[i])

In [6]:
current_time = datetime.datetime.now()
print("Current date and time:", current_time)

Current date and time: 2023-08-12 19:10:46.339160


In [7]:
## BE CAREFUL TO CHANGE THIS VARIABLE EVERY TIME
nRun = 3 # IF IN DOUBT CHANGE
# CHANGE CHANGE CHANGE CHANGE CHANGE CHANGE CHANGE CHANGE CHANGE CHANGE
# CHANGE CHANGE CHANGE CHANGE CHANGE CHANGE CHANGE CHANGE CHANGE CHANGE

np.savez(f'shuffleRun{nRun}.npz', F_early = allSIs[0], F_late = allSIs[1], R_early = allSIs[2]\
                , R_late = allSIs[3])